<a href="https://colab.research.google.com/github/agungfirdaus717-ux/torentotgd/blob/main/Torrent_to_Google_Drive_DownloaderLangsung_banyak_(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚀 Torrent to Google Drive Downloader
📒Notebook Final (Fix dengan lbry-libtorrent)

In [ ]:

# ============================
# Step 1 – Install libtorrent
# ============================
!sudo apt-get update -y
!sudo apt-get install -y python3.10 python3.10-distutils

# Install pip untuk Python 3.10
!curl -sS https://bootstrap.pypa.io/get-pip.py | python3.10

# Install libtorrent versi LBRY (support Colab)
!python3.10 -m pip install --upgrade pip setuptools wheel
!python3.10 -m pip install lbry-libtorrent

# Tes libtorrent
!python3.10 -c "import libtorrent as lt; print('✅ libtorrent version:', lt.version)"


In [ ]:

# ============================
# STEP 2: Mount Google Drive
# ============================
from google.colab import drive
drive.mount('/content/drive')

save_path = "/content/drive/MyDrive/torrent_downloads"
print("📂 Download path:", save_path)


upload torent yang mau di download Ke My drive/torrent_downloads/torrents
kalo gak ada tinggal buat foldernya

kalo magnet upload ke My drive/torrent_download format magnets.txt (tiap link magnet 1 baris)

(link khusus untuk saya, kalian login ke google drive kalian sendiri lalu ikuti langkah di atas)Upload Torent File ke sini user 0 [link](https://drive.google.com/drive/u/0/folders/1tGYAwzrBPnjucm6rcPXqDMKWePQeL18e)
magnets.txt ke sini [link ](https://drive.google.com/drive/u/0/folders/1eCfIXBosUzw4wRDMS7WnEfSUOw9hZ5Eu)

In [ ]:
# ============================
# Step 3 – Buat script torrent
# ============================
code = f"""
import libtorrent as lt
import time, os

save_path = r"{save_path}"
os.makedirs(save_path, exist_ok=True)

print("✅ libtorrent version:", lt.version)

ses = lt.session()
ses.listen_on(6881, 6891)

# ===============================
# Bisa campuran: file .torrent + magnet link
# ===============================

torrent_folder = os.path.join(save_path, "torrents")
os.makedirs(torrent_folder, exist_ok=True)

magnet_file = os.path.join(save_path, "magnets.txt")

# Ambil semua file .torrent
torrent_files = []
if os.path.exists(torrent_folder):
    torrent_files = [os.path.join(torrent_folder, f)
                     for f in os.listdir(torrent_folder)
                     if f.endswith(".torrent")]

# Ambil semua magnet link dari file txt
magnet_links = []
if os.path.exists(magnet_file):
    with open(magnet_file, "r") as f:
        magnet_links = [line.strip() for line in f if line.strip().startswith("magnet:")]

# Gabungkan semua sumber
torrent_sources = torrent_files + magnet_links

handles = []

for src in torrent_sources:
    if src.startswith("magnet:"):
        params = {{
            "save_path": save_path,
            "storage_mode": lt.storage_mode_t(2),
        }}
        handle = lt.add_magnet_uri(ses, src, params)
        print("➕ Magnet ditambahkan:", src[:60] + "...")
    else:
        info = lt.torrent_info(src)
        params = {{
            "save_path": save_path,
            "ti": info
        }}
        handle = ses.add_torrent(params)
        print("📂 Torrent file ditambahkan:", info.name())
    handles.append(handle)

print(f"✅ Total {{len(handles)}} torrent berhasil dimulai.\\n")

# ===============================
# Loop progres dengan tampilan tabel
# ===============================
while True:
    all_done = True

    # Clear layar (Windows/Linux/Mac)
    os.system("cls" if os.name == "nt" else "clear")

    print("📊 Status Torrent:")
    print("="*70)
    for h in handles:
        s = h.status()
        state_str = [
            "queued", "checking", "downloading metadata",
            "downloading", "finished", "seeding", "allocating", "checking fastresume"
        ]
        print(f"📥 {{s.name[:25]:25}} | "
              f"{{s.progress*100:6.2f}}% | "
              f"↓ {{s.download_rate/1000:6.1f}} kB/s | "
              f"peers: {{s.num_peers:3d}} | "
              f"state: {{state_str[s.state]}}")

        if not s.is_seeding:
            all_done = False

    if all_done:
        print("\\n🎉 Semua torrent selesai diunduh.")
        break

    time.sleep(2)
"""

with open("torrent_runner.py", "w") as f:
    f.write(code)

print("✅ File torrent_runner.py berhasil dibuat dengan tampilan tabel live.")


In [ ]:
# ============================
# Step 4 – Jalankan script
# ============================
!python3.10 torrent_runner.py
